In [248]:
import os
import sys
from tweepy import API
from tweepy import OAuthHandler
from tweepy import Cursor
import json
from collections import Counter
import math
import time
import pandas as pd
import csv
import numpy as np
from functools import reduce

In [251]:
def getTwitterOAuth():
    consumer_key = 'MmGB3E6Vwhv0IxidlmktTMPls'
    consumer_secret = 'xPs6HQxgPi6z91XMWHyBmQH2WVron3Gbve1PddLRRDUWtjXCdC'
    access_token = '797785657070485504-0QTz3l4Y50Ko8EPZVJWKtpk3MFzd9vO'
    access_token_secret = 'iwuVBltOVFGfWHoPmCFgfIWqN2IcS8iNXMTyUVdU2Vj5Z'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token,access_token_secret)
    return auth

def TwitterClient():
    auth = getTwitterOAuth()
    client = API(auth)
    return client

#global variables
MAX_FRIENDS = 15000
max_pages = math.ceil(MAX_FRIENDS/5000)
        
def makeDirectory():
    client = TwitterClient()
    #dirname = "users/{}".format(user)
    dirname = "users"
    
    try:
        os.makedirs(dirname, mode =0o755, exist_ok=True)
    except OSError:
        print("Directory {} already exist".format(dirname))
    except Exception as e:
        print( "Error while creating directory {}".format(dirname))
        print(e)
        sys.exit(1)

        
        
        
        
def getHashtags(tweet):
    entities = tweet.get('entities',{})
    hashtags = entities.get('hashtags',[])
    return [tag['text'].lower() for tag in hashtags]

def tags(user='AvokaAyebilla'):
    hashtags = Counter()
    taglist = []
    fname = "users/influencers/{}_timeline.jsonl".format(user)
    with open(fname, 'r') as f:
        for line in f:
            tweet = json.loads(line)
            tags_in_tweet = getHashtags(tweet)
            hashtags.update(tags_in_tweet)
        return {user:hashtags.most_common(5)} #gets the five most used hashtag

#Gets a user's timeline, followers count, likes, retweets, tweet_count
def getUserTweets(user='AvokaAyebilla'):
    #makeDirectory()
    ufollowers = 0
    uretweets = 0
    ulikes = 0
    utweets = 0
    ufriends = 0
    utweet = 0
    umentions = []
    fname = "users/influencers/{}_timeline.jsonl".format(user)
    if not os.path.exists(os.path.join('users', "/influencers/{}_timeline.jsonl".format(user))):
        client = TwitterClient()
        with open(fname, 'w') as f:
            pages = Cursor(client.user_timeline, screen_name=user,count=200).pages(max_pages)
            for apage in pages:
                for twet in apage:
                    ufollowers = twet.user.followers_count
                    ufriends =   twet.user.friends_count
                    utweet =     twet.user.statuses_count
                    break
                break

            for page in pages:
                for status in page:
                    f.write(json.dumps(status._json) + "\n")
                    status = status._json
                    uretweets = uretweets + status['retweet_count']
                    ulikes =  ulikes + status['favorite_count']
                    mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
                    umentions.append(mentions)
                    
                if len(page) == 800:
                    time.sleep(60) 
    else:
        
        timeline = "users/influencers/{}_timeline.jsonl".format(user)
        with open(timeline, 'r') as f:
            for line in f:
                twet = json.loads(line)
                ufollowers =twet['user']['followers_count']
                ufriends =  twet['user']['friends_count']
                utweet = twet['user']['statuses_count']
                break
            
            for line in f:
                twet = json.loads(line)
                mentions = ", ".join([mention['screen_name'] for mention in twet['entities']['user_mentions']])
                umentions.append(mentions)
                uretweets = uretweets + twet['retweet_count']
                ulikes =  ulikes + twet['favorite_count']
                    
    return {user:[ufollowers, uretweets,ulikes,utweet,ufriends,len(umentions),uretweets+ulikes,ufollowers-ufriends,len(umentions)]}

        
  #reads saved list of users from csv file on disk into python list
def worker():
    uList = []
    tagList = []
    
    #Open one data file at a time to generate stats for the two groups
    with open('African_influencers.csv', 'rt') as f:
    #with open('Top_gov_officials.csv', 'rt') as f:
        f.readline()
        csv_reader = csv.reader(f)
        for line in csv_reader:
            try:
                uList.append(getUserTweets(line[0]))
                #tagList.append(tags(line[0]))
            except Exception as e:
                print( "Error while getting timeline for  {}".format(line[0]))
                print(e)
    return uList,tagList


#converts the list of dictionaries into a dictionary
def ConvertToDict(ListDictLists):
    NewDict = {}  
    for listItem in ListDictLists:
        for key, value in listItem.items(): 
            if key in list(NewDict):  # if the key already exists, append to new
                for entry in value:
                    NewDict[key].append(entry)
            else:  
                NewDict[key] = value
    return NewDict

#returns the keys of the dictionary as a list
def get_dictkeys(listdic):
    return list (listdic.keys())

#returns the values of the dictionary as a list
def get_dictvals(listdic):
     return list(listdic.values())
    
def dataframe():
    uList, tagList= worker()
    dictt = ConvertToDict(uList)
    data =  get_dictvals(dictt)
    df = pd.DataFrame(data, columns = ["followers", "retweets","likes","tweets","friends","mentions","popularity_score","reach_score","relevance_score"])
    coll = get_dictkeys(dictt)
    df.insert(0,'Influencers',coll)
    print(tagList)
    return df
#dff = dataframe()
df_influ = dataframe()
#dff



Error while getting timeline for  andBeyondSafari
Twitter error response: status code = 404
Error while getting timeline for  hurricanevaness
Twitter error response: status code = 404
Error while getting timeline for  BBCKarenAllen
Twitter error response: status code = 404
Error while getting timeline for  drewfhinshaw
Twitter error response: status code = 404
Error while getting timeline for  Afrinnovator
Twitter error response: status code = 401
Error while getting timeline for  The_New_Age
Twitter error response: status code = 401
Error while getting timeline for  702JohnRobbie
Twitter error response: status code = 404
[]


In [230]:
plrtsort = dff.sort_values(by=['popularity_score'])

In [231]:
rchtsort = dff.sort_values(by=['reach_score'])

In [233]:
rlvtsort = dff.sort_values(by=['relevance_score'])

In [239]:
reach = rchtsort.tail(10)

In [240]:
relevant = rlvtsort.tail(10)

In [241]:
popular = plrtsort.tail(10)

In [249]:
reduce(np.intersect1d, [reach.Influencers, relevant.Influencers,popular.Influencers])

array(['PaulKagame'], dtype=object)

In [252]:
df_influ

,Influencers,followers,retweets,likes,tweets,friends,mentions,popularity_score,reach_score,relevance_score
0,gettleman,25703,82120,8645,3773,37,398,90765,25666,398
1,a24media,31284,4891,708,16869,3059,400,5599,28225,400
2,andiMakinana,101306,290473,8366,142236,2838,400,298839,98468,400
3,AfricaCheck,68073,2832,2985,27327,4592,400,5817,63481,400
4,JamesCopnall,21965,56366,687,19439,5049,395,57053,16916,395
...,...,...,...,...,...,...,...,...,...,...
88,Julius_S_Malema,3131066,464275,119928,37220,651,399,584203,3130415,399
89,News24,3579245,14824,29136,322954,632,400,43960,3578613,400
90,SAPresident,18,0,0,19,14,0,0,4,0
91,GarethCliff,1974685,216883,21847,31627,356,398,238730,1974329,398
